In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive')

Drive already mounted at /content/MyDrive; to attempt to forcibly remount, call drive.mount("/content/MyDrive", force_remount=True).


In [ ]:
!pip install transformers
!pip3 install kobert-transformers

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from kobert_transformers import get_kobert_model, get_tokenizer

In [ ]:
LABEL2NUM = {
    '310300200':0,
    '310300100':1,
    '310300300':2,
    '310300400':3,
    '310300600':4,
    '310300500':5,
    '310260800':6,
    '310260700':7,
    '310260600':8,
    '310260500':9,
    '310260400':10,
    '310260100':11,
    '310260200':12,
    '310260300':13,
    '310150080':14,
    '310150010':15,
    '310150030':16,
    '310150090':17,
    '310150040':18,
    '310150070':19,
    '310130030':20,
    '310130080':21,
    '310130040':22,
    '310120030':23,
    '310120110':24,
    '310120020':25,
    '310250':26,
    '310400100':27,
    '310400200':28,
    '310200200':29,
    '310200100':30,
    '310220300':31,
    '310220100':32,
    '310220200':33,
    '320300300':34,
    '320300200':35,
    '320300100':36,
    '320300400':37,
    '320300600':38,
    '320300500':39,
    '320210600':40,
    '320210500':41,
    '320210700':42,
    '320210400':43,
    '320210100':44,
    '320210200':45,
    '320210300':46,
    '320120600':47,
    '320120100':48,
    '320120200':49,
    '320120300':50,
    '320120700':51,
    '320400':52,
    '320500100':53,
    '320500200':54,
    '320160700':55,
    '320160800':56,
    '320180700':57,
    '320180600':58,
    '405100':59,
    '405300300':60,
    '405300100':61,
    '405300200':62,
    '405200300':63,
    '405200400':64,
    '405200100':65,
    '405200200':66,
    '405400200':67,
    '405400300':68,
    '405400400':69,
    '405400500':70,
    '405400100':71,
    '405400600':72,
    '430100400':73,
    '430100100':74,
    '430100200':75,
    '430100300':76,
    '430100500':77,
    '430200100':78,
    '430200300':79,
    '430200200':80,
    '430200500':81,
    '430200400':82,
    '430300100':83,
    '430400':84,
    '430500':85,
    '421200100':86,
    '421200200':87,
    '421200300':88,
    '421200400':89,
    '421200500':90,
    '421100100':91,
    '421100200':92,
    '421100300':93,
    '421100400':94,
    '421100500':95,
    '421300100':96,
    '422200100':97,
    '422200200':98,
    '422200300':99,
    '422200400':100,
    '422200500':101,
    '422200600':102,
    '422200700':103,
    '422100100':104,
    '422100200':105,
    '422100300':106,
    '422100400':107,
    '422100500':108,
    '422100600':109,
    '422300100':110,
    '422300200':111,
    '422300300':112,
    '422400100':113,
    '422400200':114,
    '422500100':115,
    '422500200':116,
    '422500300':117,
    '422500400':118,
    '422500500':119,
    '422600100':120,
    '422600200':121,
    '400070100':122,
    '400070700':123,
    '400070200':124,
    '400070500':125,
    '400080300':126,
    '400080200':127,
    '400120100':128,
    '400120200':129,
    '400130100':130,
    '400130200':131,
    '400110100':132,
    '400110200':133,
    '400140100':134,
    '400140200':135,
    '400090':136,
    '400600':137
}

LABEL2NAME = {
    '310300200':{'대분류':'여성의류', '중분류':'아우터', '소분류':'패딩'},
    '310300100':{'대분류':'여성의류', '중분류':'아우터', '소분류':'점퍼'},
    '310300300':{'대분류':'여성의류', '중분류':'아우터', '소분류':'코트'},
    '310300400':{'대분류':'여성의류', '중분류':'아우터', '소분류':'자켓'},
    '310300600':{'대분류':'여성의류', '중분류':'아우터', '소분류':'가디건'},
    '310300500':{'대분류':'여성의류', '중분류':'아우터', '소분류':'조끼/베스트'},
    '310260800':{'대분류':'여성의류', '중분류':'상의', '소분류':'니트/스웨터'},
    '310260700':{'대분류':'여성의류', '중분류':'상의', '소분류':'후드티/후드집업'},
    '310260600':{'대분류':'여성의류', '중분류':'상의', '소분류':'맨투맨'},
    '310260500':{'대분류':'여성의류', '중분류':'상의', '소분류':'블라우스'},
    '310260400':{'대분류':'여성의류', '중분류':'상의', '소분류':'셔츠'},
    '310260100':{'대분류':'여성의류', '중분류':'상의', '소분류':'반팔 티셔츠'},
    '310260200':{'대분류':'여성의류', '중분류':'상의', '소분류':'긴팔 티셔츠'},
    '310260300':{'대분류':'여성의류', '중분류':'상의', '소분류':'민소매 티셔츠'},
    '310150080':{'대분류':'여성의류', '중분류':'바지', '소분류':'데님/청바지'},
    '310150010':{'대분류':'여성의류', '중분류':'바지', '소분류':'슬랙스'},
    '310150030':{'대분류':'여성의류', '중분류':'바지', '소분류':'면바지'},
    '310150090':{'대분류':'여성의류', '중분류':'바지', '소분류':'반바지'},
    '310150040':{'대분류':'여성의류', '중분류':'바지', '소분류':'트레이닝/조거팬츠'},
    '310150070':{'대분류':'여성의류', '중분류':'바지', '소분류':'레깅스'},
    '310130030':{'대분류':'여성의류', '중분류':'치마', '소분류':'롱 스커트'},
    '310130080':{'대분류':'여성의류', '중분류':'치마', '소분류':'미디 스커트'},
    '310130040':{'대분류':'여성의류', '중분류':'치마', '소분류':'미니 스커트'},
    '310120030':{'대분류':'여성의류', '중분류':'원피스', '소분류':'롱 원피스'},
    '310120110':{'대분류':'여성의류', '중분류':'원피스', '소분류':'미디 원피스'},
    '310120020':{'대분류':'여성의류', '중분류':'원피스', '소분류':'미니 원피스'},
    '310250':{'대분류':'여성의류', '중분류':'점프수트'},
    '310400100':{'대분류':'여성의류', '중분류':'셋업/세트', '소분류':'정장/셋업'},
    '310400200':{'대분류':'여성의류', '중분류':'셋업/세트', '소분류':'트레이닝/스웨트 셋업'},
    '310200200':{'대분류':'여성의류', '중분류':'언더웨어/홈웨어', '소분류':'홈웨어'},
    '310200100':{'대분류':'여성의류', '중분류':'언더웨어/홈웨어', '소분류':'언더웨어'},
    '310220300':{'대분류':'여성의류', '중분류':'테마/이벤트', '소분류':'코스튬/코스프레'},
    '310220100':{'대분류':'여성의류', '중분류':'테마/이벤트', '소분류':'한복'},
    '310220200':{'대분류':'여성의류', '중분류':'테마/이벤트', '소분류':'드레스'},
    '320300300':{'대분류':'남성의류', '중분류':'아우터', '소분류':'패딩'},
    '320300200':{'대분류':'남성의류', '중분류':'아우터', '소분류':'점퍼'},
    '320300100':{'대분류':'남성의류', '중분류':'아우터', '소분류':'코트'},
    '320300400':{'대분류':'남성의류', '중분류':'아우터', '소분류':'자켓'},
    '320300600':{'대분류':'남성의류', '중분류':'아우터', '소분류':'가디건'},
    '320300500':{'대분류':'남성의류', '중분류':'아우터', '소분류':'조끼/베스트'},
    '320210600':{'대분류':'남성의류', '중분류':'상의', '소분류':'후드티/후드집업'},
    '320210500':{'대분류':'남성의류', '중분류':'상의', '소분류':'맨투맨'},
    '320210700':{'대분류':'남성의류', '중분류':'상의', '소분류':'니트/스웨터'},
    '320210400':{'대분류':'남성의류', '중분류':'상의', '소분류':'셔츠'},
    '320210100':{'대분류':'남성의류', '중분류':'상의', '소분류':'반팔 티셔츠'},
    '320210200':{'대분류':'남성의류', '중분류':'상의', '소분류':'긴팔 티셔츠'},
    '320210300':{'대분류':'남성의류', '중분류':'상의', '소분류':'민소매 티셔츠'},
    '320120600':{'대분류':'남성의류', '중분류':'바지', '소분류':'데님/청바지'},
    '320120100':{'대분류':'남성의류', '중분류':'바지', '소분류':'면바지'},
    '320120200':{'대분류':'남성의류', '중분류':'바지', '소분류':'슬랙스'},
    '320120300':{'대분류':'남성의류', '중분류':'바지', '소분류':'트레이닝/조거팬츠'},
    '320120700':{'대분류':'남성의류', '중분류':'바지', '소분류':'반바지'},
    '320400':{'대분류':'남성의류', '중분류':'점프수트'},
    '320500100':{'대분류':'남성의류', '중분류':'셋업/세트', '소분류':'정장/셋업'},
    '320500200':{'대분류':'남성의류', '중분류':'셋업/세트', '소분류':'트레이닝/스웨트 셋업'},
    '320160700':{'대분류':'남성의류', '중분류':'언더웨어/홈웨어', '소분류':'언더웨어'},
    '320160800':{'대분류':'남성의류', '중분류':'언더웨어/홈웨어', '소분류':'홈웨어'},
    '320180700':{'대분류':'남성의류', '중분류':'테마/이벤트', '소분류':'코스튬/코스프레'},
    '320180600':{'대분류':'남성의류', '중분류':'테마/이벤트', '소분류':'한복'},
    '405100':{'대분류':'신발', '중분류':'스니커즈'},
    '405300300':{'대분류':'신발', '중분류':'남성화', '소분류':'샌들/슬리퍼'},
    '405300100':{'대분류':'신발', '중분류':'남성화', '소분류':'구두/로퍼'},
    '405300200':{'대분류':'신발', '중분류':'남성화', '소분류':'워커/부츠'},
    '405200300':{'대분류':'신발', '중분류':'여성화', '소분류':'샌들/슬리퍼'},
    '405200400':{'대분류':'신발', '중분류':'여성화', '소분류':'구두'},
    '405200100':{'대분류':'신발', '중분류':'여성화', '소분류':'단화/플랫슈즈'},
    '405200200':{'대분류':'신발', '중분류':'여성화', '소분류':'워커/부츠'},
    '405400200':{'대분류':'신발', '중분류':'스포츠화', '소분류':'농구화'},
    '405400300':{'대분류':'신발', '중분류':'스포츠화', '소분류':'골프화'},
    '405400400':{'대분류':'신발', '중분류':'스포츠화', '소분류':'축구/풋살화'},
    '405400500':{'대분류':'신발', '중분류':'스포츠화', '소분류':'테니스화'},
    '405400100':{'대분류':'신발', '중분류':'스포츠화', '소분류':'등산화/트레킹화'},
    '405400600':{'대분류':'신발', '중분류':'스포츠화', '소분류':'야구화'},
    '430100400':{'대분류':'가방/지갑', '중분류':'여성가방', '소분류':'클러치백'},
    '430100100':{'대분류':'가방/지갑', '중분류':'여성가방', '소분류':'숄더백'},
    '430100200':{'대분류':'가방/지갑', '중분류':'여성가방', '소분류':'크로스백'},
    '430100300':{'대분류':'가방/지갑', '중분류':'여성가방', '소분류':'토트백'},
    '430100500':{'대분류':'가방/지갑', '중분류':'여성가방', '소분류':'백팩'},
    '430200100':{'대분류':'가방/지갑', '중분류':'남성가방', '소분류':'클러치'},
    '430200300':{'대분류':'가방/지갑', '중분류':'남성가방', '소분류':'숄더백'},
    '430200200':{'대분류':'가방/지갑', '중분류':'남성가방', '소분류':'크로스백'},
    '430200500':{'대분류':'가방/지갑', '중분류':'남성가방', '소분류':'브리프케이스'},
    '430200400':{'대분류':'가방/지갑', '중분류':'남성가방', '소분류':'백팩'},
    '430300100':{'대분류':'가방/지갑', '중분류':'남성가방', '소분류':'캐리어'},
    '430400':{'대분류':'가방/지갑', '중분류':'여성지갑'},
    '430500':{'대분류':'가방/지갑', '중분류':'남성지갑'},
    '421200100':{'대분류':'시계', '중분류':'남성시계', '소분류':'프리미엄 시계(디지털)'},
    '421200200':{'대분류':'시계', '중분류':'남성시계', '소분류':'프리미엄 시계(쿼츠)'},
    '421200300':{'대분류':'시계', '중분류':'남성시계', '소분류':'프리미엄 시계(오토매틱)'},
    '421200400':{'대분류':'시계', '중분류':'남성시계', '소분류':'일반 시계(메탈 밴드)'},
    '421200500':{'대분류':'시계', '중분류':'남성시계', '소분류':'일반 시계(가죽 밴드)'},
    '421100100':{'대분류':'시계', '중분류':'여성시계', '소분류':'프리미엄 시계(디지털)'},
    '421100200':{'대분류':'시계', '중분류':'여성시계', '소분류':'프리미엄 시계(쿼츠)'},
    '421100300':{'대분류':'시계', '중분류':'여성시계', '소분류':'프리미엄 시계(오토매틱)'},
    '421100400':{'대분류':'시계', '중분류':'여성시계', '소분류':'일반 시계(메탈 밴드)'},
    '421100500':{'대분류':'시계', '중분류':'여성시계', '소분류':'일반 시계(가죽 밴드)'},
    '421300100':{'대분류':'시계', '중분류':'시계용품', '소분류':'스트랩'},
    '422200100':{'대분류':'쥬얼리', '중분류':'귀걸이/피어싱', '소분류':'다이아몬드 귀걸이'},
    '422200200':{'대분류':'쥬얼리', '중분류':'귀걸이/피어싱', '소분류':'금 귀걸이'},
    '422200300':{'대분류':'쥬얼리', '중분류':'귀걸이/피어싱', '소분류':'은 귀걸이'},
    '422200400':{'대분류':'쥬얼리', '중분류':'귀걸이/피어싱', '소분류':'진주/유색보석 귀걸이'},
    '422200500':{'대분류':'쥬얼리', '중분류':'귀걸이/피어싱', '소분류':'패션 귀걸이'},
    '422200600':{'대분류':'쥬얼리', '중분류':'귀걸이/피어싱', '소분류':'피어싱'},
    '422200700':{'대분류':'쥬얼리', '중분류':'귀걸이/피어싱', '소분류':'귀찌/이어커프'},
    '422100100':{'대분류':'쥬얼리', '중분류':'목걸이/펜던트', '소분류':'다이아몬드 목걸이'},
    '422100200':{'대분류':'쥬얼리', '중분류':'목걸이/펜던트', '소분류':'금 목걸이'},
    '422100300':{'대분류':'쥬얼리', '중분류':'목걸이/펜던트', '소분류':'은 목걸이'},
    '422100400':{'대분류':'쥬얼리', '중분류':'목걸이/펜던트', '소분류':'진주/유색보석 목걸이'},
    '422100500':{'대분류':'쥬얼리', '중분류':'목걸이/펜던트', '소분류':'패션 목걸이'},
    '422100600':{'대분류':'쥬얼리', '중분류':'목걸이/펜던트', '소분류':'초커'},
    '422300100':{'대분류':'쥬얼리', '중분류':'팔찌', '소분류':'금팔찌'},
    '422300200':{'대분류':'쥬얼리', '중분류':'팔찌', '소분류':'은팔찌'},
    '422300300':{'대분류':'쥬얼리', '중분류':'팔찌', '소분류':'패션 팔찌'},
    '422400100':{'대분류':'쥬얼리', '중분류':'발찌', '소분류':'금발찌'},
    '422400200':{'대분류':'쥬얼리', '중분류':'발찌', '소분류':'패션발찌'},
    '422500100':{'대분류':'쥬얼리', '중분류':'반지', '소분류':'다이아몬드 반지'},
    '422500200':{'대분류':'쥬얼리', '중분류':'반지', '소분류':'금반지'},
    '422500300':{'대분류':'쥬얼리', '중분류':'반지', '소분류':'은반지'},
    '422500400':{'대분류':'쥬얼리', '중분류':'반지', '소분류':'진주/유색보석 반지'},
    '422500500':{'대분류':'쥬얼리', '중분류':'반지', '소분류':'패션반지'},
    '422600100':{'대분류':'쥬얼리', '중분류':'쥬얼리 세트', '소분류':'귀금속 쥬얼리 세트'},
    '422600200':{'대분류':'쥬얼리', '중분류':'쥬얼리 세트', '소분류':'패션 쥬얼리 세트'},
    '400070100':{'대분류':'패션 액세서리', '중분류':'모자', '소분류':'볼캡'},
    '400070700':{'대분류':'패션 액세서리', '중분류':'모자', '소분류':'버킷'},
    '400070200':{'대분류':'패션 액세서리', '중분류':'모자', '소분류':'스냅백'},
    '400070500':{'대분류':'패션 액세서리', '중분류':'모자', '소분류':'비니'},
    '400080300':{'대분류':'패션 액세서리', '중분류':'안경/선글라스', '소분류':'선글라스'},
    '400080200':{'대분류':'패션 액세서리', '중분류':'안경/선글라스', '소분류':'안경'},
    '400120100':{'대분류':'패션 액세서리', '중분류':'목도리/장갑', '소분류':'목도리'},
    '400120200':{'대분류':'패션 액세서리', '중분류':'목도리/장갑', '소분류':'장갑'},
    '400130100':{'대분류':'패션 액세서리', '중분류':'스카프/넥타이', '소분류':'스카프'},
    '400130200':{'대분류':'패션 액세서리', '중분류':'스카프/넥타이', '소분류':'넥타이'},
    '400110100':{'대분류':'패션 액세서리', '중분류':'벨트', '소분류':'남성 벨트'},
    '400110200':{'대분류':'패션 액세서리', '중분류':'벨트', '소분류':'여성 벨트'},
    '400140100':{'대분류':'패션 액세서리', '중분류':'양말/스타킹', '소분류':'양말'},
    '400140200':{'대분류':'패션 액세서리', '중분류':'양말/스타킹', '소분류':'스타킹'},
    '400090':{'대분류':'패션 액세서리', '중분류':'우산/양산'},
    '400600':{'대분류':'패션 액세서리', '중분류':'키링/키케이스'}
}

In [ ]:
from transformers.models.auto.modeling_auto import MODEL_FOR_VISION_2_SEQ_MAPPING
num_classes = 138
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model1 (KoBERT)
kobert_model = get_kobert_model()
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 138,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate is not None:
            self.dropout = nn.Dropout(p=dr_rate)
        else:
            self.dropout = None
    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(input_ids=input_ids,
                                    attention_mask=attention_mask,
                                    return_dict=False)
        output = self.dropout(pooled_output)
        output = self.classifier(output)
        return output
model1 = BERTClassifier(kobert_model, num_classes=num_classes, dr_rate=0.5).to(device)
save_path = "/content/MyDrive/MyDrive/model/KoBERT/model_label1.pth"
model1.load_state_dict(torch.load(save_path))

# model2 (MobileNetv2)
import torch
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torchvision import models

model2 = models.mobilenet_v2(pretrained=False)
model2.classifier[-1] = nn.Linear(model2.last_channel, num_classes)
model2 = model2.to(device)
model2.load_state_dict(torch.load('/content/MyDrive/MyDrive/model/MobileNetv2/[ver.4.2] fine_tuned_mobilenetv2.pth'))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

In [ ]:
import torch.nn.functional as F

class EnsembleModel(nn.Module):
  def __init__(self, model1, model2):
    super(EnsembleModel, self).__init__()
    self.model1 = model1
    self.model2 = model2

  def forward(self, input_ids, attention_mask, image):
    output1 = self.model1(input_ids=input_ids, attention_mask=attention_mask)
    output2 = self.model2(image)

    # softmax
    # output1 = F.softmax(output1, dim=1)
    # output2 = F.softmax(output2, dim=1)

    # 평균
    weighted_output = output1 * 2 + output2
    weighted_output = F.softmax(weighted_output, dim=1)

    top3 = torch.topk(weighted_output.data, 3, dim=1)

    output_idx = top3.indices.to('cpu').numpy()
    output_score = top3.values.to('cpu').numpy()

    NUM2LABEL = {v:k for k,v in LABEL2NUM.items()}

    pred_cat_id = [NUM2LABEL[idx] for idx in output_idx[0]]
    pred_cat = [LABEL2NAME[idx] for idx in pred_cat_id]

    result = list(zip(pred_cat_id, pred_cat, output_score[0]))

    return result

In [ ]:
import cv2
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
    ToTensorV2()
    ])

class EnsembleDataset(Dataset):
    def __init__(self,
                 sentence,
                 image,
                 bert_tokenizer,
                 max_len,
                 pad,
                 pair,
                 transforms=transform):
        self.sentence = sentence
        self.image = image
        self.tokenizer = bert_tokenizer
        self.max_len = max_len
        self.pad = pad
        self.pair = pair
        self.transforms = transform

    def __getitem__(self, index):

        # text
        inputs = self.tokenizer.encode_plus(
                 self.sentence,
                 None if not self.pair else self.sentence,
                 add_special_tokens=True,
                 max_length=self.max_len,
                 padding='max_length' if self.pad else None,
                 truncation=True
        )

        input_ids = torch.tensor(inputs['input_ids'], dtype=torch.long)
        attention_mask = torch.tensor(inputs['attention_mask'], dtype=torch.long)

        # image
        image = self.transforms(image=self.image)['image']

        return input_ids, attention_mask, image

## 앙상블 테스트

In [ ]:
tokenizer = get_tokenizer()
img_path = '/content/226536015-310260800 (2).jpg'
image = cv2.imread(img_path)
data_ensemble = EnsembleDataset('남성 티셔츠',
                                image,
                                tokenizer,
                                64,
                                True,
                                False,
                                transform)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [ ]:
model = EnsembleModel(model1, model2)
input_ids, attention_mask, image = data_ensemble[0]
input_ids = input_ids.unsqueeze(dim=0)
input_ids = input_ids.to(device)

attention_mask = attention_mask.unsqueeze(dim=0)
attention_mask = attention_mask.to(device)

image = image.unsqueeze(dim=0)
image = image.to(device)

attention_mask = attention_mask.to(device)
image = image.to(device)
final_output = model(input_ids, attention_mask, image)

In [ ]:
final_output

[('320210400', {'대분류': '남성의류', '중분류': '상의', '소분류': '셔츠'}, 0.8475373),
 ('320210200', {'대분류': '남성의류', '중분류': '상의', '소분류': '긴팔 티셔츠'}, 0.14588201),
 ('320160800', {'대분류': '남성의류', '중분류': '언더웨어/홈웨어', '소분류': '홈웨어'}, 0.0027812014)]

In [ ]:
result = predict('구스다운패딩(웨스트우드)90 거의 새상품', '199552910-310300200 (1).jpg', model)
result

[('310300200', {'대분류': '여성의류', '중분류': '아우터', '소분류': '패딩'}, 0.99999976),
 ('320300300', {'대분류': '남성의류', '중분류': '아우터', '소분류': '패딩'}, 6.251792e-08),
 ('310300400', {'대분류': '여성의류', '중분류': '아우터', '소분류': '자켓'}, 5.8437166e-08)]

In [ ]:
result = predict('폼폼푸린 25주년운동화 키링', '147849656-400600.jpg', model)
result

[('400600', {'대분류': '패션 액세서리', '중분류': '키링/키케이스'}, 0.99999154),
 ('422100500',
  {'대분류': '쥬얼리', '중분류': '목걸이/펜던트', '소분류': '패션 목걸이'},
  7.386477e-06),
 ('430100400', {'대분류': '가방/지갑', '중분류': '여성가방', '소분류': '클러치백'}, 6.1597046e-07)]

In [ ]:
result = predict('데님 셔츠 캐주얼 청남방 빈티지', '225260434-310260400 (1).jpg', model)
result

[('310260400', {'대분류': '여성의류', '중분류': '상의', '소분류': '셔츠'}, 0.9996437),
 ('320210400', {'대분류': '남성의류', '중분류': '상의', '소분류': '셔츠'}, 0.0003554327),
 ('310300400', {'대분류': '여성의류', '중분류': '아우터', '소분류': '자켓'}, 6.6591815e-07)]

In [ ]:
result = predict('프라이탁 블레어 그레이 흰 민트 새상품', '223087099-430500 (1).jpg', model)
result

[('430100200', {'대분류': '가방/지갑', '중분류': '여성가방', '소분류': '크로스백'}, 0.7693644),
 ('430200200', {'대분류': '가방/지갑', '중분류': '남성가방', '소분류': '크로스백'}, 0.14543986),
 ('430200300', {'대분류': '가방/지갑', '중분류': '남성가방', '소분류': '숄더백'}, 0.06502811)]

In [ ]:
result = predict('디디에두보 14k 브라운다이아 귀걸이', '182150060-422200100 (1).jpg', model)
result

[('422200100',
  {'대분류': '쥬얼리', '중분류': '귀걸이/피어싱', '소분류': '다이아몬드 귀걸이'},
  0.97434014),
 ('310150090', {'대분류': '여성의류', '중분류': '바지', '소분류': '반바지'}, 0.024116505),
 ('310260800', {'대분류': '여성의류', '중분류': '상의', '소분류': '니트/스웨터'}, 0.0011769187)]

In [ ]:
result = predict('에이치에이트 H8 슬랙스 (오트밀) ABRAMS WOOL PANT', '219396323-310150010 (1).jpg', model)
result

[('320120200', {'대분류': '남성의류', '중분류': '바지', '소분류': '슬랙스'}, 0.9533069),
 ('310150010', {'대분류': '여성의류', '중분류': '바지', '소분류': '슬랙스'}, 0.04435452),
 ('310400100', {'대분류': '여성의류', '중분류': '셋업/세트', '소분류': '정장/셋업'}, 0.0022309904)]

In [ ]:
result = predict('브이 귀걸이 피어싱', '224821249-422200600 (1).jpg', model)
result

[('422200600', {'대분류': '쥬얼리', '중분류': '귀걸이/피어싱', '소분류': '피어싱'}, 0.9810358),
 ('310400100', {'대분류': '여성의류', '중분류': '셋업/세트', '소분류': '정장/셋업'}, 0.010415015),
 ('320300400', {'대분류': '남성의류', '중분류': '아우터', '소분류': '자켓'}, 0.0029928891)]

In [ ]:
result = predict('원형 리본 비즈키링', '226471067-400600 (1).jpg', model)
result

[('400600', {'대분류': '패션 액세서리', '중분류': '키링/키케이스'}, 0.99701715),
 ('422100500',
  {'대분류': '쥬얼리', '중분류': '목걸이/펜던트', '소분류': '패션 목걸이'},
  0.0029511342),
 ('422600200',
  {'대분류': '쥬얼리', '중분류': '쥬얼리 세트', '소분류': '패션 쥬얼리 세트'},
  1.3697046e-05)]

In [ ]:
result = predict('(택포) MAGPIEROBIN 나시가디건 세트', '226534681-310300600 (1).jpg', model)
result

[('320300600', {'대분류': '남성의류', '중분류': '아우터', '소분류': '가디건'}, 0.98794365),
 ('310300600', {'대분류': '여성의류', '중분류': '아우터', '소분류': '가디건'}, 0.012056163),
 ('310400100', {'대분류': '여성의류', '중분류': '셋업/세트', '소분류': '정장/셋업'}, 7.426179e-08)]

In [ ]:
result = predict('[정품] 롤렉스 여성용 데이저스트 화이트로만 다이얼 풀세트', '220566857-421100300 (1).jpg', model)
result

[('310400100', {'대분류': '여성의류', '중분류': '셋업/세트', '소분류': '정장/셋업'}, 0.42754212),
 ('421200400',
  {'대분류': '시계', '중분류': '남성시계', '소분류': '일반 시계(메탈 밴드)'},
  0.29409072),
 ('421100300',
  {'대분류': '시계', '중분류': '여성시계', '소분류': '프리미엄 시계(오토매틱)'},
  0.14466977)]

## 테스트 데이터 성능 평가

In [ ]:
import cv2

def predict(text, image, model):

  data_ensemble = EnsembleDataset(text,
                                  image,
                                  tokenizer,
                                  64,
                                  True,
                                  False,
                                  transform)
  input_ids, attention_mask, image = data_ensemble[0]
  input_ids = input_ids.unsqueeze(dim=0)
  input_ids = input_ids.to(device)

  attention_mask = attention_mask.unsqueeze(dim=0)
  attention_mask = attention_mask.to(device)

  image = image.unsqueeze(dim=0)
  image = image.to(device)

  final_output = model(input_ids, attention_mask, image)

  return final_output

In [ ]:
import pandas as pd
test_data = pd.read_csv('/content/MyDrive/MyDrive/test_data_preprocessed.csv')
test_data['image_paths'] = test_data['image_paths'].map(eval)
test_data['len'] = test_data['image_paths'].map(len)
test_data = test_data[test_data['len'] != 0]
test_data.reset_index(drop=True, inplace=True)
test_data.head()

,prd_id,name,image_paths,cat_id,len
0,199552910-310300200,구스다운패딩(웨스트우드)90 거의 새상품,"[199552910-310300200 (1).jpg, 199552910-310300...",310300200,4
1,208906996-310300200,퀄팅 다이아 패딩 여자 패딩점퍼,"[208906996-310300200 (2).jpg, 208906996-310300...",310300200,2
2,225858509-310300200,K2패딩,"[225858509-310300200 (1).jpg, 225858509-310300...",310300200,7
3,207651040-310300200,아이더 스투키 숏패딩 남녀공용 블랙 s,"[207651040-310300200 (1).jpg, 207651040-310300...",310300200,5
4,212490892-310300200,게스 항공 패딩,[212490892-310300200.jpg],310300200,1


In [ ]:
# 이미지 데이터 불러오기
import zipfile
zip_path = '/content/MyDrive/MyDrive/test_image_resized.zip'
extract_path = '/content/test_image_resized'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
  zip_ref.extractall(extract_path)

In [ ]:
from tqdm import tqdm
for i in tqdm(range(len(test_data))):
  text = test_data.loc[i, 'name']
  image_path = test_data.loc[i, 'image_paths'][0]
  image = cv2.imread('/content/test_image_resized/' + image_path)

  pred = predict(text, image, model)

  test_data.loc[i, 'pred'] = str(pred)

100%|██████████| 12418/12418 [06:44<00:00, 30.71it/s]


In [ ]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12418 entries, 0 to 12417
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   prd_id       12418 non-null  object
 1   name         12418 non-null  object
 2   image_paths  12418 non-null  object
 3   cat_id       12418 non-null  int64 
 4   len          12418 non-null  int64 
 5   pred         12418 non-null  object
dtypes: int64(2), object(4)
memory usage: 582.2+ KB


In [ ]:
test_data['cat_id'] = test_data['cat_id'].astype('str')
test_data['pred'] = test_data['pred'].map(eval)

In [ ]:
# top3
test_data['pred_cat_top3'] = test_data['pred'].apply(lambda x: [pred[0] for pred in x])
test_data['score_top3'] = test_data.apply(lambda x: 1 if x['cat_id'] in x['pred_cat_top3'] else 0, axis=1)
print(test_data['score_top3'].sum() / len(test_data))

0.8443388629408922


In [ ]:
# top1
test_data['pred_cat_top1'] = test_data['pred'].apply(lambda x: x[0][0])
test_data['score_top1'] = test_data.apply(lambda x: 1 if x['cat_id'] == x['pred_cat_top1'] else 0, axis=1)
print(test_data['score_top1'].sum() / len(test_data))

0.6735384119826059
